In [2]:
import pandas as pd 
import numpy as np 

In [3]:
beneficiary = pd.read_csv('data/clean/medtrend_for_sql.csv')
patients = pd.read_csv('data/clean/telemeduse_for_sql.csv')
service_count = pd.read_csv('data/clean/tele_service_for_sql.csv')

print(beneficiary.shape)
print(patients.shape)
print(service_count.shape)

(23685, 8)
(3292, 6)
(3874, 7)


In [4]:
# Check for missing values
print("Missing values in Beneficiary dataset:")
print(beneficiary.isna().sum())

print("\nMissing values in Patients dataset:")
print(patients.isna().sum())

print("\nMissing values in ServiceCount dataset:")
print(service_count.isna().sum())

Missing values in Beneficiary dataset:
row_id                   0
Year-Quarter             0
State                    0
Bene_Sex_Desc            0
Bene_Age_Desc            0
Total_Bene_TH_Elig       0
Total_Bene_Telehealth    0
Pct_Telehealth           0
dtype: int64

Missing values in Patients dataset:
row_id            0
Year-Quarter      0
Group             0
State             0
Subgroup          0
Value           188
dtype: int64

Missing values in ServiceCount dataset:
Year-Quarter                0
State                       0
Year                        0
Month                       0
TelehealthType              0
ServiceCount                0
RatePer1000Beneficiaries    0
dtype: int64


In [5]:

# Impute missing values in 'Value' column with -1
patients['Value'].fillna(-1, inplace=True)

/var/folders/tx/7d0rdqt930q3m1lsb2vvbfj40000gn/T/ipykernel_41222/125188313.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  patients['Value'].fillna(-1, inplace=True)
